# For cleaning the data to export into one .csv for use in R Shiny app

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import statsmodels.formula.api as smf 

#### import data

In [2]:
happiness = pd.read_csv('../data/hapiscore_whr.csv') # happiness score
fish = pd.read_csv('../data/fisfod_cons_pc.csv') # fish consumption per capita
sugar = pd.read_csv('../data/sugar_per_person_g_per_day.csv') # g per day
continents = pd.read_csv('../data/continents.csv') 
disorders = pd.read_csv('../data/depressive_anxiety_IHME.csv') # depressive and anxiety disorders

#### many countries have different ways they're referred to in the datasets, dict to use for renaming to common convention

In [3]:
# Dict for keeping names consistent across sets
name_changes = (
    {'Hong Kong SAR, China':'Hong Kong',
     'Macao SAR, China':'Macao',
     'Korea, Rep.':'South Korea',
     'Korea, Dem. Rep.':'North Korea',
     'Czechia':'Czech Republic',
     'Slovak Republic':'Slovakia',
     'Kyrgyz Republic':'Kyrgyzstan',
     'Moldova, Republic of':'Moldova',
     'Tanzania, United Republic of':'Tanzania',
     'United Republic of Tanzania':'Tanzania',
     'UK':'United Kingdom',
     'USA':'United States',
     'United States of America':'United States',
     'Russian Federation':'Russia',
     'Congo, Dem. Rep.':'Democratic Republic of the Congo',
     'Congo, Rep.':'Congo',
     'Lao PDR':'Laos',
     "Lao People's Democratic Republic":'Laos',
     'Bahamas, The':'Bahamas',
     'Brunei Darussalam':'Brunei',
     'Congo (Democratic Republic of the)': 'Democratic Republic of the Congo',
     "Cote d'Ivoire": "Ivory Coast",
     "Côte d'Ivoire": "Ivory Coast",
     'Gambia, The': 'Gambia',
     'The Gambia': 'Gambia',
     'Iran, Islamic Rep.': 'Iran',
     'Iran (Islamic Republic of)':'Iran',
     "Korea, Dem. People's Rep.": 'North Korea',
     "Democratic People's Republic of Korea": 'North Korea',
     'Micronesia, Fed. Sts.': 'Micronesia',
     'Turkey':'Turkiye',
     'Turks and Caicos Islands':'Turks and Caicos',
     'Syrian Arab Republic':'Syria',
     'Venezuela, RB':'Venezuela',
     'Venezuela (Bolivarian Republic of)':'Venezuela',
     'Viet Nam':'Vietnam',
     'Yemen, Rep.':'Yemen',
     'Curacao':'Curaçao',
     'Burma (Myanmar)': 'Myanmar',
     'Dem. Rep. Congo': 'Democratic Republic of the Congo',
     'Korea, North': 'North Korea',
     'Korea, South': 'South Korea',
     "Republic of Korea": 'South Korea',
     'Sint Maarten (Dutch part)': 'Sint Maarten',
     'St. Martin (French part)': 'Sint Maarten',
     'Swaziland': 'Eswatini',
     'Bahamas, The': 'Bahamas',
     'The Bahamas': 'Bahamas',
     'Egypt, Arab Rep.': 'Egypt',
     "Bolivia (Plurinational State of)": 'Bolivia',
     "Republic of Moldova": 'Moldova',
     "American Samoa": 'Samoa',
     "Lao": 'Laos',
     'Taiwan (Province of China)': 'Taiwan',
     'Türkiye':'Turkiye',
     'UAE': 'United Arab Emirates',
     }
)

## Cleaning and organizing data to create .csv for export to R

In [4]:
# melt happiness from wide to long
happiness_long = happiness.melt(id_vars=['country'], var_name='year', value_name='happiness_score')
happiness_long['year'] = pd.to_numeric(happiness_long['year'])
happiness_long['country'] = happiness_long['country'].replace(name_changes)

fish_long = fish.melt(id_vars=['country'], var_name='year', value_name='pcp_fish_consumption')
fish_long['year'] = pd.to_numeric(fish_long['year'])
fish_long['country'] = fish_long['country'].replace(name_changes)

sugar_long = sugar.melt(id_vars=['country'], var_name='year', value_name='sugar_per_cap')
sugar_long['year'] = pd.to_numeric(sugar_long['year'])
sugar_long['country'] = sugar_long['country'].replace(name_changes)

disorders_long = disorders[['location', 'cause', 'year', 'val']].rename(columns={'location':'country', 'cause':'disorder', 'val':'pct_new_per_pop'}).replace(name_changes)
disorders_long['year'] = pd.to_numeric(disorders_long['year'])
disorders_long['country'] = disorders_long['country'].replace(name_changes)
# combine depressive disorders and anxiety disorders into one column 
disorders_long = disorders_long.groupby(['country', 'year']).sum().reset_index()
disorders_long = disorders_long.rename(columns={'pct_new_per_pop':'pct_new_per_pop_disorders'})[['country', 'year', 'pct_new_per_pop_disorders']]


In [5]:
disorders_long

,country,year,pct_new_per_pop_disorders
0,Afghanistan,1990,0.013309
1,Afghanistan,1991,0.013174
2,Afghanistan,1992,0.013054
3,Afghanistan,1993,0.012866
4,Afghanistan,1994,0.012437
...,...,...,...
6491,Zimbabwe,2017,0.005582
6492,Zimbabwe,2018,0.005676
6493,Zimbabwe,2019,0.005708
6494,Zimbabwe,2020,0.006104


In [6]:
# merge dfs on country and year
happy_fish = pd.merge(happiness_long, fish_long, on=['country', 'year'], how='inner').dropna()
happy_sugar_fish = pd.merge(happy_fish, sugar_long, on=['country', 'year'], how='inner').dropna()
happy_sad_sugar_fish = pd.merge(happy_sugar_fish, disorders_long, on=['country', 'year'], how='inner').dropna()
happy_sad_sugar_fish = happy_sad_sugar_fish.rename(
    columns={'sugar_per_cap':'sugar_g_per_person_per_day',
             'pcp_fish_consumption':'fish_kg_per_person_per_year'}
             )
happy_sad_sugar_fish

,country,year,happiness_score,fish_kg_per_person_per_year,sugar_g_per_person_per_day,pct_new_per_pop_disorders
0,Australia,2005,73.4,25.30,128.0,0.015860
1,Belgium,2005,72.6,24.40,146.0,0.011938
2,Brazil,2005,66.4,6.04,115.0,0.012405
3,Canada,2005,74.2,23.60,147.0,0.008716
4,Czech Republic,2005,64.4,10.00,103.0,0.013115
...,...,...,...,...,...,...
1595,Vietnam,2018,53.0,37.30,33.7,0.005701
1596,Yemen,2018,30.6,3.10,82.4,0.012322
1597,South Africa,2018,48.8,6.35,113.0,0.008865
1598,Zambia,2018,40.4,11.70,31.4,0.006512


In [7]:
# see if countries are the same in each df
print(f'Count of countries in each:\n\n {happy_sugar_fish["country"].nunique(), disorders_long["country"].nunique()}\n\n')

# see which countries are different - Countries in life_expectancy but not in happy_forest_sugar_fish_merged
print(f'in happy_sugar_fish, not in disorders_long:\n\n {np.setdiff1d(happy_sugar_fish["country"].unique(), disorders_long["country"].unique())}\n\n')

# see which countries are different - Countries in continents but not in life_expectancy
print(f'in disorders_long, not in happy_sugar_fish:\n\n {np.setdiff1d(disorders_long["country"].unique(), happy_sugar_fish["country"].unique())}\n\n')

Count of countries in each:

 (151, 203)


in happy_sugar_fish, not in disorders_long:

 ['Hong Kong, China']


in disorders_long, not in happy_sugar_fish:

 ['Andorra' 'Antigua and Barbuda' 'Bahamas' 'Bahrain' 'Barbados' 'Bermuda'
 'Bhutan' 'Brunei' 'Burundi' 'Cabo Verde' 'Comoros' 'Cook Islands'
 'Democratic Republic of the Congo' 'Dominica' 'Equatorial Guinea'
 'Eritrea' 'Fiji' 'Greenland' 'Grenada' 'Guam' 'Guinea-Bissau' 'Kiribati'
 'Libya' 'Marshall Islands' 'Micronesia (Federated States of)' 'Monaco'
 'Nauru' 'Niue' 'North Korea' 'Northern Mariana Islands' 'Palau'
 'Palestine' 'Papua New Guinea' 'Puerto Rico' 'Qatar'
 'Saint Kitts and Nevis' 'Saint Lucia' 'Saint Vincent and the Grenadines'
 'Samoa' 'San Marino' 'Sao Tome and Principe' 'Seychelles' 'Singapore'
 'Solomon Islands' 'Somalia' 'South Sudan' 'Syria' 'Timor-Leste' 'Tokelau'
 'Tonga' 'Tuvalu' 'United States Virgin Islands' 'Vanuatu']




In [8]:
happy_sad_sugar_fish.to_csv('../data/happy_sad_sugar_fish.csv', index=False)

## Exploration of this data - mostly the newer disorders data

In [9]:
happy_sad_sugar_fish    

,country,year,happiness_score,fish_kg_per_person_per_year,sugar_g_per_person_per_day,pct_new_per_pop_disorders
0,Australia,2005,73.4,25.30,128.0,0.015860
1,Belgium,2005,72.6,24.40,146.0,0.011938
2,Brazil,2005,66.4,6.04,115.0,0.012405
3,Canada,2005,74.2,23.60,147.0,0.008716
4,Czech Republic,2005,64.4,10.00,103.0,0.013115
...,...,...,...,...,...,...
1595,Vietnam,2018,53.0,37.30,33.7,0.005701
1596,Yemen,2018,30.6,3.10,82.4,0.012322
1597,South Africa,2018,48.8,6.35,113.0,0.008865
1598,Zambia,2018,40.4,11.70,31.4,0.006512


In [10]:
fig = px.scatter(happy_sad_sugar_fish,
           x='year',
           y='pct_new_per_pop_disorders',
           color='country',
           title='Depressive and Anxiety Disorders by Country and Year',
           template='plotly_dark')

# toggle visibility off by default
for trace in fig.data:
    trace.visible = 'legendonly'
    
fig.show()

In [11]:
fig = px.scatter(happy_sad_sugar_fish,
           x='sugar_per_cap',
           y='pct_new_per_pop_disorders',
           color='country',
           title='Depressive and Anxiety Disorders by Country and Year',
           template='plotly_dark',
           trendline = 'ols', 
           trendline_color_override='red')

# toggle visibility off by default
for trace in fig.data:
    trace.visible = 'legendonly'
    
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['country', 'year', 'happiness_score', 'fish_kg_per_person_per_year', 'sugar_g_per_person_per_day', 'pct_new_per_pop_disorders'] but received: sugar_per_cap